In [2]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from transformers import AdamW
from torch.nn import MSELoss
from collections import Counter
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

/Users/ormeiri/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ormeiri/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
df = pd.read_csv('/Users/ormeiri/Desktop/Work/sci-search/parser/data/combined_data_new.csv')
df

,term_id,langs,country,term,type,code_lang,hl,gl,Collection date,Result number,...,Is there reference to everyday life?,Is there reference to everyday life? פרופורציה,Are there local examples in the content? (e.g. relevant case study in a country or ethnic group that speaks the language of the content),Are there local examples in the content? (e.g. relevant case study in a country or ethnic group that speaks the language of the content)פרופורציה,Does the content present advantages and disadvantages or risks and benefits?,Does the content present advantages and disadvantages or risks and benefits? פרופורציה,ssi_mean,SSI mean פרופורציה Integrated,raw_html,extracted_text
0,70,Zulu,"Cape Town,Western Cape,South Africa",Isikhukhula,Socio-Scientific Issues,zu,zu,za,20.2.2023,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,NaN,"<html>\r\n<head>\r\n<META NAME=""robots"" CONTEN...",NaN
1,71,Zulu,"Cape Town,Western Cape,South Africa",Isomiso,Socio-Scientific Issues,zu,zu,za,20.2.2023,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,NaN,\n \n\n\n\n\n\n\n\n\n<!DOCTYPE html>\n<html x...,\ndrought\n
2,71,Zulu,"Cape Town,Western Cape,South Africa",Isomiso,Socio-Scientific Issues,zu,zu,za,20.2.2023,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,NaN,"<!doctype html><html lang=""en""><head><script>v...",NaN
3,29,Zulu,"Cape Town,Western Cape,South Africa",Isisindo,Canonical Scientific Issues,zu,zu,za,20.2.2023,11.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,NaN,"<!DOCTYPE html><html style=""font-size: 10px;fo...",NaN
4,29,Zulu,"Cape Town,Western Cape,South Africa",Isisindo,Canonical Scientific Issues,zu,zu,za,20.2.2023,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,NaN,"<!DOCTYPE html><html style=""font-size: 10px;fo...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4156,56,Ukrainian,"Kyiv,Kyiv city,Ukraine",калорія,Socio-Scientific Issues,uk,uk,ua,22.12.2022,4.0,...,2.0,1.0,1.0,0.5,2.0,1.0,0.833333,0.833333,"<!DOCTYPE html>\n<html lang=""uk"">\n<head> \n\...",Фітнес портал Калорійність продуктів — як її п...
4157,58,Ukrainian,"Kyiv,Kyiv city,Ukraine",відновлювальна енергія,Socio-Scientific Issues,uk,uk,ua,22.12.2022,2.0,...,2.0,1.0,1.0,0.5,2.0,1.0,0.833333,0.833333,"<!DOCTYPE html><html lang=""uk""><head><meta cha...","Відновлювану енергію можна отримувати з води, ..."
4158,58,Ukrainian,"Kyiv,Kyiv city,Ukraine",відновлювальна енергія,Socio-Scientific Issues,uk,uk,ua,22.12.2022,3.0,...,2.0,1.0,2.0,1.0,2.0,1.0,1.000000,1.000000,"<!DOCTYPE html><html style=""font-size: 10px;fo...",NaN
4159,12,Ukrainian,"Kyiv,Kyiv city,Ukraine",гриб,Canonical Scientific Issues,uk,uk,ua,22.12.2022,8.0,...,2.0,1.0,2.0,1.0,2.0,1.0,1.000000,NaN,"<!DOCTYPE html>\n<html xmlns=""http://www.w3.or...",У лісі поблизу Ворохти виявили гігантський біл...


In [4]:
def clean_text(text):
    text = str(text)
    cleaned_text = re.sub(r'[^\w\s]', '', text)
    return cleaned_text

In [5]:
df['cleaned_text'] = df['extracted_text'].apply(clean_text)
X = df[df['cleaned_text'] != 'nan']
X

,term_id,langs,country,term,type,code_lang,hl,gl,Collection date,Result number,...,Is there reference to everyday life? פרופורציה,Are there local examples in the content? (e.g. relevant case study in a country or ethnic group that speaks the language of the content),Are there local examples in the content? (e.g. relevant case study in a country or ethnic group that speaks the language of the content)פרופורציה,Does the content present advantages and disadvantages or risks and benefits?,Does the content present advantages and disadvantages or risks and benefits? פרופורציה,ssi_mean,SSI mean פרופורציה Integrated,raw_html,extracted_text,cleaned_text
1,71,Zulu,"Cape Town,Western Cape,South Africa",Isomiso,Socio-Scientific Issues,zu,zu,za,20.2.2023,1.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,NaN,\n \n\n\n\n\n\n\n\n\n<!DOCTYPE html>\n<html x...,\ndrought\n,\ndrought\n
5,35,Zulu,"Cape Town,Western Cape,South Africa",amaplanethi azungeza ilanga,Canonical Scientific Issues,zu,zu,za,20.2.2023,1.0,...,0.5,1.0,0.5,1.0,0.5,0.500000,0.500000,"<!DOCTYPE html><html lang=""zu"" dir=""ltr""><head...",Isikhala kusuka ku-A kuya ku-Z Ngaphandle kwe-...,Isikhala kusuka kuA kuya kuZ Ngaphandle kweMer...
6,43,Zulu,"Cape Town,Western Cape,South Africa",Ulibofuzo,Novel Science and Technology Issues,zu,zu,za,20.2.2023,2.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,NaN,"<!doctype html>\n <html lang=""zu"">\n <he...","Nkosikhona Cele | June 11, 2022 SITHOLAKALE ...",Nkosikhona Cele June 11 2022 SITHOLAKALE si...
8,54,Zulu,"Cape Town,Western Cape,South Africa",Ukuba nokudla,Socio-Scientific Issues,zu,zu,za,20.2.2023,3.0,...,1.0,0.0,0.0,0.0,0.0,0.333333,NaN,"\r\n\r\n<!DOCTYPE html PUBLIC ""-//W3C//DTD XHT...","I-College of Agriculture, Engineering and Scie...",ICollege of Agriculture Engineering and Scienc...
9,56,Zulu,"Cape Town,Western Cape,South Africa",Ikhalori,Socio-Scientific Issues,zu,zu,za,20.2.2023,3.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,NaN,"<!DOCTYPE html>\n<html ⚡ lang=""en-US"">\n<head>...","Omunye izitsha athandwa kakhulu, okuyinto, ka...",Omunye izitsha athandwa kakhulu okuyinto kany...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4155,56,Ukrainian,"Kyiv,Kyiv city,Ukraine",калорія,Socio-Scientific Issues,uk,uk,ua,22.12.2022,3.0,...,1.0,2.0,1.0,2.0,1.0,1.000000,1.000000,"<!doctype html><html lang=""uk""><head><meta...","\nПрирода влаштувала нас так, що, навіть з`ївш...",\nПрирода влаштувала нас так що навіть зївши з...
4156,56,Ukrainian,"Kyiv,Kyiv city,Ukraine",калорія,Socio-Scientific Issues,uk,uk,ua,22.12.2022,4.0,...,1.0,1.0,0.5,2.0,1.0,0.833333,0.833333,"<!DOCTYPE html>\n<html lang=""uk"">\n<head> \n\...",Фітнес портал Калорійність продуктів — як її п...,Фітнес портал Калорійність продуктів як її пр...
4157,58,Ukrainian,"Kyiv,Kyiv city,Ukraine",відновлювальна енергія,Socio-Scientific Issues,uk,uk,ua,22.12.2022,2.0,...,1.0,1.0,0.5,2.0,1.0,0.833333,0.833333,"<!DOCTYPE html><html lang=""uk""><head><meta cha...","Відновлювану енергію можна отримувати з води, ...",Відновлювану енергію можна отримувати з води в...
4159,12,Ukrainian,"Kyiv,Kyiv city,Ukraine",гриб,Canonical Scientific Issues,uk,uk,ua,22.12.2022,8.0,...,1.0,2.0,1.0,2.0,1.0,1.000000,NaN,"<!DOCTYPE html>\n<html xmlns=""http://www.w3.or...",У лісі поблизу Ворохти виявили гігантський біл...,У лісі поблизу Ворохти виявили гігантський біл...


In [6]:
def normalize_text(text):
    # Regex to match dates in various formats
    date_patterns = [
        r'\b\d{1,2}[\/.-]\d{1,2}[\/.-]\d{2,4}\b',  # Numeric dates like 19/09/2024, 25.12.2012
        r'\b\d{4}[\/.-]\d{1,2}[\/.-]\d{1,2}\b',  # Numeric dates like 2024-09-19
        r'\b\d{1,2}\s+(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\s+\d{4}\b',  # Dates like 15 Nov 2013, 1 January 2022
        r'\b(January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2},?\s+\d{4}\b',  # Dates like March 15, 2023 and March 15 2023
        r'\b\d{1,2}\s+(January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{4}\b',  # Dates like 15 March 2023
    ]
    # Combine all date regex patterns into one
    date_regex = '|'.join(date_patterns)

    text = re.sub(date_regex, '<date>', text)  # Replace dates with <date>
    # URLs
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '<url>', text)
    # Email addresses
    text = re.sub(r'\b[\w.-]+?@\w+?\.\w{2,4}\b', '<email>', text)
    # Money amounts
    text = re.sub(r'\$\d+(?:\.\d{2})?', '<money>', text)
    # Phone numbers
    text = re.sub(r'\b\d{3}-\d{4}\b', '<phone>', text)

    return text

In [7]:
X['cleaned_text'] = X['cleaned_text'].apply(normalize_text)
X

/var/folders/68/dmpmrx5n0ts80f_y59l1n8y00000gn/T/ipykernel_2025/3863618037.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['cleaned_text'] = X['cleaned_text'].apply(normalize_text)


,term_id,langs,country,term,type,code_lang,hl,gl,Collection date,Result number,...,Is there reference to everyday life? פרופורציה,Are there local examples in the content? (e.g. relevant case study in a country or ethnic group that speaks the language of the content),Are there local examples in the content? (e.g. relevant case study in a country or ethnic group that speaks the language of the content)פרופורציה,Does the content present advantages and disadvantages or risks and benefits?,Does the content present advantages and disadvantages or risks and benefits? פרופורציה,ssi_mean,SSI mean פרופורציה Integrated,raw_html,extracted_text,cleaned_text
1,71,Zulu,"Cape Town,Western Cape,South Africa",Isomiso,Socio-Scientific Issues,zu,zu,za,20.2.2023,1.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,NaN,\n \n\n\n\n\n\n\n\n\n<!DOCTYPE html>\n<html x...,\ndrought\n,\ndrought\n
5,35,Zulu,"Cape Town,Western Cape,South Africa",amaplanethi azungeza ilanga,Canonical Scientific Issues,zu,zu,za,20.2.2023,1.0,...,0.5,1.0,0.5,1.0,0.5,0.500000,0.500000,"<!DOCTYPE html><html lang=""zu"" dir=""ltr""><head...",Isikhala kusuka ku-A kuya ku-Z Ngaphandle kwe-...,Isikhala kusuka kuA kuya kuZ Ngaphandle kweMer...
6,43,Zulu,"Cape Town,Western Cape,South Africa",Ulibofuzo,Novel Science and Technology Issues,zu,zu,za,20.2.2023,2.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,NaN,"<!doctype html>\n <html lang=""zu"">\n <he...","Nkosikhona Cele | June 11, 2022 SITHOLAKALE ...",Nkosikhona Cele <date> SITHOLAKALE sinqunyw...
8,54,Zulu,"Cape Town,Western Cape,South Africa",Ukuba nokudla,Socio-Scientific Issues,zu,zu,za,20.2.2023,3.0,...,1.0,0.0,0.0,0.0,0.0,0.333333,NaN,"\r\n\r\n<!DOCTYPE html PUBLIC ""-//W3C//DTD XHT...","I-College of Agriculture, Engineering and Scie...",ICollege of Agriculture Engineering and Scienc...
9,56,Zulu,"Cape Town,Western Cape,South Africa",Ikhalori,Socio-Scientific Issues,zu,zu,za,20.2.2023,3.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,NaN,"<!DOCTYPE html>\n<html ⚡ lang=""en-US"">\n<head>...","Omunye izitsha athandwa kakhulu, okuyinto, ka...",Omunye izitsha athandwa kakhulu okuyinto kany...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4155,56,Ukrainian,"Kyiv,Kyiv city,Ukraine",калорія,Socio-Scientific Issues,uk,uk,ua,22.12.2022,3.0,...,1.0,2.0,1.0,2.0,1.0,1.000000,1.000000,"<!doctype html><html lang=""uk""><head><meta...","\nПрирода влаштувала нас так, що, навіть з`ївш...",\nПрирода влаштувала нас так що навіть зївши з...
4156,56,Ukrainian,"Kyiv,Kyiv city,Ukraine",калорія,Socio-Scientific Issues,uk,uk,ua,22.12.2022,4.0,...,1.0,1.0,0.5,2.0,1.0,0.833333,0.833333,"<!DOCTYPE html>\n<html lang=""uk"">\n<head> \n\...",Фітнес портал Калорійність продуктів — як її п...,Фітнес портал Калорійність продуктів як її пр...
4157,58,Ukrainian,"Kyiv,Kyiv city,Ukraine",відновлювальна енергія,Socio-Scientific Issues,uk,uk,ua,22.12.2022,2.0,...,1.0,1.0,0.5,2.0,1.0,0.833333,0.833333,"<!DOCTYPE html><html lang=""uk""><head><meta cha...","Відновлювану енергію можна отримувати з води, ...",Відновлювану енергію можна отримувати з води в...
4159,12,Ukrainian,"Kyiv,Kyiv city,Ukraine",гриб,Canonical Scientific Issues,uk,uk,ua,22.12.2022,8.0,...,1.0,2.0,1.0,2.0,1.0,1.000000,NaN,"<!DOCTYPE html>\n<html xmlns=""http://www.w3.or...",У лісі поблизу Ворохти виявили гігантський біл...,У лісі поблизу Ворохти виявили гігантський біл...


In [8]:
stop_words_set = set()
# Get unique languages from the 'langs' column
unique_langs = X['langs'].unique()  # Add more languages as needed

# Convert to Pandas Series
series_with_nan = pd.Series(unique_langs)

# Removing NaN values
cleaned_series = series_with_nan.dropna()


# Convert back to ndarray if necessary
cleaned_array = cleaned_series.to_numpy()

for lang in cleaned_array:
    try:
        stop_words_set.update(stopwords.words(lang))
    except OSError:
        print(f"No stop words found for '{lang}' in NLTK.")

No stop words found for 'Zulu' in NLTK.
No stop words found for 'Hindi' in NLTK.
No stop words found for 'Taiwanese' in NLTK.
No stop words found for 'Japanese' in NLTK.
No stop words found for 'Afrikaans' in NLTK.
No stop words found for 'Korean' in NLTK.
No stop words found for 'Vietnamese' in NLTK.
No stop words found for 'Swahili' in NLTK.
No stop words found for 'Ukrainian' in NLTK.


In [9]:
def remove_stopwords(text, stopwords):
    words = text.lower().split()  # Convert text to lowercase before splitting
    filtered_words = [word for word in words if word not in stopwords]
    return ' '.join(filtered_words)

In [10]:
# Apply the function to the cleaned_text column
X['cleaned_text'] = X['cleaned_text'].apply(lambda x: remove_stopwords(x, stop_words_set))

/var/folders/68/dmpmrx5n0ts80f_y59l1n8y00000gn/T/ipykernel_2025/1613584100.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['cleaned_text'] = X['cleaned_text'].apply(lambda x: remove_stopwords(x, stop_words_set))


In [11]:
# Initialize a Counter object to store word counts
word_counts = Counter()

# Tokenize each entry and update the Counter
for text in X['cleaned_text']:
    # Split the text into words based on spaces and update counts
    word_counts.update(text.split())
    
word_counts

Counter({'это': 66686,
         'луне': 60727,
         'луны': 43661,
         'астронавты': 33338,
         'nasa': 31783,
         'луну': 31667,
         'года': 24196,
         'земле': 23632,
         'лунной': 23395,
         'время': 22644,
         'очень': 20760,
         'полета': 20418,
         'земли': 20212,
         'астронавтов': 19841,
         'несколько': 19758,
         'лунного': 18636,
         'фотографии': 18114,
         'поверхности': 17743,
         'поэтому': 16605,
         'лунный': 16168,
         'американцы': 15755,
         'действительно': 14921,
         'apollo': 14731,
         'г': 14703,
         'лишь': 14477,
         'क': 13399,
         'двигателя': 13378,
         'полетов': 12970,
         'просто': 12707,
         'лунных': 12648,
         'которые': 12458,
         'аполлонов': 12373,
         'человек': 12253,
         'поверхность': 12113,
         'астронавтами': 11950,
         'также': 11911,
         'раза': 11795,
         'изза':

In [12]:
# Remove rows where any of the target variables are NaN
combined_df = X.dropna(subset=['quality_mean', 'accessibility_mean'])
# Create a combined DataFrame (assuming 'quality_mean', 'accessibility_mean', 'ssi_mean' are columns in df)
combined_df = combined_df[['cleaned_text', 'quality_mean', 'accessibility_mean']]
combined_df

,cleaned_text,quality_mean,accessibility_mean
1,drought,0.266667,0.333333
5,isikhala kusuka kua kuya kuz ngaphandle kwemer...,0.733333,0.638889
6,nkosikhona cele <date> sitholakale sinqunywe i...,0.600000,0.416667
8,icollege agriculture engineering science icaes...,0.333333,0.500000
9,omunye izitsha athandwa kakhulu okuyinto kanye...,0.116667,0.416667
...,...,...,...
4155,природа влаштувала що навіть зївши зайве людин...,0.566667,0.833333
4156,фітнес портал калорійність продуктів як її пра...,0.716667,0.916667
4157,відновлювану енергію можна отримувати з води в...,0.516667,0.833333
4159,лісі поблизу ворохти виявили гігантський білий...,0.400000,0.500000


# Tokenization

In [13]:
# Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
tokenized_texts = [tokenizer.tokenize(text) for text in combined_df['cleaned_text']]

In [ ]:
# Convert tokens to input IDs
input_ids = [tokenizer.convert_tokens_to_ids(tokens) for tokens in tokenized_texts]
# Padding
max_len = max(len(x) for x in input_ids)
input_ids = [x + [0]*(max_len-len(x)) for x in input_ids]

In [ ]:
# Convert to PyTorch tensors
input_ids = torch.tensor(input_ids)
labels = torch.tensor(combined_df['quality_mean'])

In [ ]:
# Train-validation-test split
train_inputs, test_inputs, train_labels, test_labels = train_test_split(input_ids, labels, test_size=0.2, random_state=42)
train_inputs, val_inputs, train_labels, val_labels = train_test_split(train_inputs, train_labels, test_size=0.2, random_state=42)

# Create DataLoaders

In [ ]:
# Create DataLoaders
train_data = TensorDataset(train_inputs, train_labels)
train_dataloader = DataLoader(train_data, batch_size=8, shuffle=True)

val_data = TensorDataset(val_inputs, val_labels)
val_dataloader = DataLoader(val_data, batch_size=8, shuffle=False)

test_data = TensorDataset(test_inputs, test_labels)
test_dataloader = DataLoader(test_data, batch_size=8, shuffle=False)

# Model training

In [ ]:
# Load pre-trained model
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)

In [ ]:
# Fine-tuning
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
epochs = 3
for epoch in range(epochs):
    model.train()
    for batch in train_dataloader:
        optimizer.zero_grad()
        inputs, labels = batch
        outputs = model(inputs)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()

# Evaluation on validation set
model.eval()
with torch.no_grad():
    val_outputs = model(val_inputs)
    val_predictions = torch.argmax(val_outputs.logits, dim=1)
    val_accuracy = (val_predictions == val_labels).float().mean()

print(f"Validation Accuracy: {val_accuracy.item()}")

In [2]:
# # Load tokenizer and model
# model_name = 'bert-base-multilingual-cased'
# tokenizer = BertTokenizer.from_pretrained(model_name)
# model = BertForSequenceClassification.from_pretrained(model_name, num_labels=1)  # num_labels=1 for regression
# 
# # Ensure model is in evaluation mode by default
# model.eval()

In [3]:
# # Check for non-string types or NaN values
# print(combined_df['cleaned_text'].apply(type).value_counts())
# print(combined_df['cleaned_text'].isnull().sum())

In [4]:
# # Load tokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
# 
# def preprocess(texts):
#     # Explicitly convert texts to a list if it isn't already one
#     if isinstance(texts, pd.Series):
#         texts = texts.tolist()
#     elif isinstance(texts, str):  # Handling the case if a single string is passed
#         texts = [texts]
#     
#     return tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
# 
# # Applying preprocessing on the 'cleaned_text' column
# X = preprocess(combined_df['cleaned_text'])
# 
# print(X.keys())
# print(X['input_ids'].shape)

In [5]:
# y = torch.tensor(combined_df['quality_mean'].values).unsqueeze(1)

In [6]:
# # Assuming df is your DataFrame with 'text' and 'quality' as columns
# train_val_df, test_df = train_test_split(X, test_size=0.15, random_state=42)
# train_df, val_df = train_test_split(train_val_df, test_size=(0.15/0.85), random_state=42)  # Adjusts to 15% of the original
# 
# # Tokenize each dataset
# train_encodings = preprocess(train_df['cleaned_text'])
# val_encodings = preprocess(val_df['cleaned_text'])
# test_encodings = preprocess(test_df['cleaned_text'])
# 
# # Convert labels to tensors
# train_labels = torch.tensor(train_df['quality'].values).unsqueeze(1)
# val_labels = torch.tensor(val_df['quality'].values).unsqueeze(1)
# test_labels = torch.tensor(test_df['quality'].values).unsqueeze(1)